# IMDb movie reviews sentiment classification

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
# from jupyterthemes import jtplot

In [ ]:
# from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import ngrams

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from keras.optimizers import SGD, Adam, Adagrad, Adadelta, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

In [ ]:
# set plot rc parameters

# jtplot.style(grid=False)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#464646'
#plt.rcParams['axes.edgecolor'] = '#FFFFFF'
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['text.color'] = '#666666'
plt.rcParams['axes.labelcolor'] = '#666666'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.color'] = '#666666'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.color'] = '#666666'
plt.rcParams['ytick.labelsize'] = 14

# plt.rcParams['font.size'] = 16

sns.color_palette('dark')
%matplotlib inline

tqdm.pandas()

## Read Data

In [ ]:
data_df = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
data_df.head()

In [ ]:
data_df.shape

In [ ]:
data_df['sentiment'].value_counts()

In [ ]:
# print 10 random reviews
for i in range(10):
    idx = np.random.randint(1, 50001)
    print('Review {}:'.format(i+1))
    print('\n\n')
    print(data_df.iloc[idx]['review'])
    print('\n')
    print('*'*100)
    print('\n')

## Text preprocessing

In [ ]:
# remove break symbol
def remove_break(text):
    return re.sub(r'<br />', '', text)

# remove punctuations
def remove_punct(text):
    nopunct = ''
    for c in text:
        if c not in string.punctuation:
            nopunct = nopunct + c
    return nopunct

# remove numbers
def remove_numbers(text):
    return re.sub(r'[0-9]', '', text)

# remove links
def remove_links(text):
    return re.sub(r'http\S+', '', text)

# remove stop words
def remove_stop_words(word_list):

    # stop words
    stopwords_list = set(stopwords.words('english'))
    # remove stop words
    word_list = [word for word in word_list if word not in stopwords_list]
    # stemming
    
    return ' '.join(word_list)

def get_root(word_list):
    ps = PorterStemmer()
    return [ps.stem(word) for word in word_list]

def clean_text(text):
    text = remove_break(text)
    text = remove_links(text)
    text = remove_numbers(text)
    text = remove_punct(text)
    word_list = word_tokenize(text)
    word_list = get_root(word_list)
    
    return ' '.join(word_list)

In [ ]:
data_df['clean_review'] = data_df['review'].progress_apply(clean_text)

In [ ]:
data_df.head()

## Split data

In [ ]:
Xtrain, Xcv, Ytrain, Ycv = train_test_split(data_df['clean_review'], data_df['sentiment'], random_state=21, test_size=0.3)

In [ ]:
Ytrain = Ytrain.map({'positive': 1, 'negative':0})
Ycv = Ycv.map({'positive': 1, 'negative':0})

## Tokenize data and get sequences

In [ ]:
# Tokenize our training data
tokenizer = Tokenizer(num_words=20000, oov_token='<UNK>')
tokenizer.fit_on_texts(Xtrain)

# word to number and vice versa map
word2num = tokenizer.word_index
num2word = {k:w for w, k in word2num.items()}

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(Xtrain)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=100)


test_sequences = tokenizer.texts_to_sequences(Xcv)
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=100)


## Initialize model

In [ ]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [ ]:
inp = Input(shape=(100, ))
x = Embedding(20000, 256, trainable=False)(inp)
x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(x)
x = Attention()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
file_path = 'model.hdf5'
ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=1)

In [ ]:
model.fit(train_padded, Ytrain, batch_size=1024, epochs=30, validation_data=(test_padded, Ycv), callbacks=[ckpt])

In [ ]:
# predict tags
Ycv_pred0 = model.predict(test_padded)
Ycv_pred = (Ycv_pred0 > 0.5).astype('int64')
# print model performance metrics
print("Accuracy :",metrics.accuracy_score(Ycv,Ycv_pred))
print("f1 score macro :",metrics.f1_score(Ycv,Ycv_pred, average = 'macro'))
print("f1 scoore micro :",metrics.f1_score(Ycv,Ycv_pred, average = 'micro'))
print("Hamming loss :",metrics.hamming_loss(Ycv,Ycv_pred))
fpr, tpr, thresh = roc_curve(Ycv, Ycv_pred0)
print("auc: ", auc(fpr,tpr))
print("Classification report: \n", metrics.classification_report(Ycv, Ycv_pred))

fig, ax = plt.subplots(figsize=[10,7])
ax.set_title('Receiver Operating Characteristic trainning')
ax.plot(fpr,tpr,sns.xkcd_rgb['greenish cyan'])
ax.plot([0,1],[0,1],ls='--', c=sns.xkcd_rgb['red pink'])
ax.set_xlim([-0.01,1.01])
ax.set_ylim([-0.01,1.01])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')